## Logs and hash maps

* Strictly speaking, a **log** is any append-only sequence of records. Logs may be human-readable, or they may only be machine-readable.
* This is distinct from application logs, which are a specific type of log. The two concepts are sometimes conflated however.
* Logs are a fundamental structure in a database. Logs are fast because it's hard to beat the performance of simply appending to a file.
* Another fundamental structure is the **index**. An index is a data structure which speeds up read operations. It usually slows down writes however because it requires performing work on every insert or append operation.


* The **hash map** is a simple but effective index structure.
* Both Python and JavaScript objects are ultimately just implementations of hash maps. So are the `dict` and `Map` classes in these languages.
* Databases using logs for primary storage are said to be using **log-structured storage**.
* If you combine a log and a hash map, you have a database! Thus the simplest possible practical (you can technically omit the index, but then a read operation would require a full file scan!) database: an append-only file store with an in-memory hash map index pointing to the byte offsets of records of interest.
* [Bitcask](https://en.wikipedia.org/wiki/Bitcask) is basically this, and it's a backing store for a number of more advanced architectures, as well as a database option in and of itself.
* This architecture has lots of advantages:
  * Low read and write latency, with predictable timing. Reading is just a single disc read, writing is a single disc write.
  * High throughput.
  * Backup is as easy as copying the file.
  * Straightforward error recovery. Only the last element being written can get corrupted.

* Of course, it has disadvantages as well:
  * The index must fit in memory (otherwise performance plummets).
  * Range queries get no speed-up (which e.g. time-series data would benefit from).
 
 
* If log-files are left untouched, they will grow to infinity in size.
* In practice you can split your log into *segments* of a certain size, and then periodically **merge** and **compact** those segments into a merged log.
* The compactor iterates through the log file and, for every key written to, finds the most recent value written. These operations are kept in the log, and as part of the merge all previous writes are discarded.


* There are some additional considerations when implementing log files.
* You will want to **compress** the log using a binary encoding scheme.
* A special value called a **tombstone** needs to be appended to the log in the case of key deletion.
* You can recover from a volatile memory crash in the hash map by reading from the start of the log segments, but this can take a long time if the segments are large. You can speed this up by keeping a snapshot of the hash map backed up to the disc.
* You can recover from a database write crash by calculating a checksum on the data segment being written ahead of time, and making sure, when recovering, that the check of the data included in the last write matches your checksum.
* Log files are sequential append-only, so they support multiple reads, and hence concurrent requests, if you want to service that.


* If we insert data into the log in sorted order we can maintain a much smaller log by overwriting prior value assignments in-place. Why don't we do that?
* Mainly because insertion in the middle of a file is a **random write** operation, while appending to the end of the file is a **sequential write** operation.
* In a random write we seek-write-seek-write.
* In a sequential write we seek once, then write multiple times (append from the same cursor).
* The latter is always faster than the former as disc seeks are extremely expensive. Since writing to the log has to happen every single transaction, this time difference matters a lot, and makes log-writing the more attractive option.
* Additionally, the simplicity of the log file makes concurrency and crash recovery much easier to implement.

## SSTables and LSM-trees

* The **SSTable** (Sorted String Table) is a more complex log-structured storage implementation.
* In SSTables we still have log segments, but their contents are now sorted by key.
* This design employs a balanced in-memory data structure, in this context referred to as the **memtable**. The memtable needs to retain key sort order.
* Every time a write or modify operation occurs, the data is written to the memtable.
* The memtable is periodically flushed in key order into a new log segment. This is still a sequential write!
* Those segments are periodically compacted and merged. The difference is that since the segment keys are in sorted order, this process is greatly expedited: we no longer need to scan through every value in every segment, as we can skip over stale key modifications completely!
* Because we no longer write to the log right away, and instead buffer using an in-memory data structure, we must keep an additional unordered log file in play, one that gets written to whenever the memtable gets written to. This is so that the memtable can be repopulated in the event of a crash.


* One advantage of this design is, as previously mentioned, that compacting segments is now much more efficient.
* Another advantage is that you no longer retain a complete index in memory. So this design works with large databases whose hash table index wouldn't fit in memory.
* If all data values were the same size, we could look up values by performing a binary search. However, in practice they are not.
* A sparse in-memory index is retained. At read time this index is queried for the range of memory values a database element could be located at, by finding the immediately smaller and larger keys in the index and taking the range of values in between their endpoints. That range is then scanned to find the value.
* For maximum read efficiency, keep the ranges down to the hardware page size.
* The process of actually scanning the data, once you have the pointer, is extremely fast.
* One more practical modification comes from the observation that with current hardware, disc I/O is more precious than CPU cycles.
* So most databases compress the index address space reference blocks. This makes the data smaller and leads to less time spent reading from persistent storage to volatile memory, but also some CPU cycles expended on decompression.


* Detour note: the memtable in the SSTables implementation needs to be a data structure that affords random insertion order and sorted read order. There are tree structures that have good performance for these tasks, namely red-black trees and AVL trees. I implement these myself in other notebooks in this repo.


* This arrangement is what LevelDB and RocksDB, among other databases, use.
* Cassandra and HBase use a similar scheme derived from the Google Bigtable paper.
* Lucerne, a full-text search indexing engine used by Elasticsearch and Solr, uses a similar method for storing its (much more complicated) term dictionary.


* For sufficiently small amounts of data, a log-structured hash table database is faster.
* But SSTables has several advantages. The most important ones being much larger practical memory limits, due to the sparse in-memory structure, and intrinsic support for range queries.
* There are many potential performance tuning measures that can be taken.
* For example, the SSTables architecture can be slow when looking up keys that do not exist in the database, or which have not been written to in a long time, because walking back the memtable and the list of SSTables takes time.
* You can use a **Bloom filter** (note: implemented in another notebook) to speed this process up. This algorithm is great for approximating set contents.